<font size="6"> **Portfolio Optimization** </font>

In [1]:
import numpy as np
import pandas as pd
import time

import pickle

In [2]:
from mle_quant_utils import quant_helper, mle_utils

In [3]:
from zipline.data import bundles
from zipline.utils.calendars import get_calendar
from zipline.pipeline.factors import AverageDollarVolume
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import Returns, SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

from zipline.pipeline.factors import CustomFactor, DailyReturns, Returns, SimpleMovingAverage, AnnualizedVolatility
from zipline.pipeline.data import USEquityPricing

from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar


from zipline.data.data_portal import DataPortal

In [4]:
import yaml
import os

# Retrieve parameters from configuration file
with open("../conf.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

BUNDLE_FOLDER = cfg['quantopian']['dataset3']['bundle_folder']
BUNDLE_NAME = cfg['quantopian']['dataset3']['bundle_name']
SECTOR_FOLDER = cfg['quantopian']['dataset3']['sector_folder']
SECTOR_DATA = cfg['quantopian']['dataset3']['sector_data']
SECTOR_NAMES = cfg['quantopian']['dataset3']['sector_names']

# Specify the bundle path
bundle_path = os.path.join(os.getcwd(), '..', 'data', BUNDLE_FOLDER)
sector_path = os.path.join(os.getcwd(), '..', 'data', SECTOR_FOLDER, SECTOR_DATA)
sector_file = os.path.join(os.getcwd(), '..', 'data', SECTOR_FOLDER, SECTOR_NAMES)
os.path.isdir(bundle_path)

True

In [5]:
RISK_MODEL_VERSION = 'v01'
MLALPHA_MODEL_VERSION = 'v01'
print('Risk Model version:', RISK_MODEL_VERSION, 'ML-Alpha model version:', MLALPHA_MODEL_VERSION)

Risk Model version: v01 ML-Alpha model version: v01


In [6]:
INPATH_ALPHA =  "../data/" + cfg['output']['main'] + "/" + cfg['output']['alpha_factors_ml']['folder'] + "/" + MLALPHA_MODEL_VERSION +"/"
INPATH_RISK = "../data/" + cfg['output']['main'] + "/" + cfg['output']['risk_factors']['folder'] + "/" + RISK_MODEL_VERSION +"/"

In [7]:
MODEL_ALPHA = cfg['output']['alpha_factors_ml']['model']
INFILE1 = cfg['output']['alpha_factors_ml']['features']
INFILE2 = cfg['output']['alpha_factors_ml']['targets']
INFILE3 = 'factor_returns.csv'

In [8]:
split_col = cfg['models']['alpha_ml']['split_col']
transcost_adv_win = 30
transcost_adv_impute =  1e4

# Load Data

## Risk Model Data

In [9]:
risk_model = {}
risk_files = ['idiosyncratic_var_vector', 'factor_var_vector' ,'factor_returns', 'factor_betas']
for file in risk_files:
    risk_model[file] = pd.read_csv(INPATH_RISK + file + '.csv', index_col=0)
    

In [10]:
pd.concat([risk_model['factor_betas'].head(2), risk_model['factor_betas'].tail(2)], axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
asset,,,,,,,,,,,,,,,,,,,,
A,-0.042628,0.014983,0.012869,0.003801,-0.022948,0.010030,-0.004048,-0.006718,-0.034471,0.024555,-0.017108,0.018875,0.025266,0.014909,-0.026406,0.033897,-0.029173,-0.030546,-0.026341,-0.012199
AAL,-0.060402,0.258601,-0.037516,0.144527,0.296615,-0.080756,0.113555,-0.024766,-0.161712,-0.035469,0.043421,0.100238,-0.014771,-0.026967,0.015362,0.025396,-0.013467,-0.056342,0.067339,0.092211
ZION,-0.061633,-0.007099,0.011137,0.132754,0.003718,0.084665,-0.047824,0.025166,0.045630,-0.035794,0.018539,-0.011147,-0.012005,-0.025919,0.020266,0.001111,-0.002798,0.019725,0.009674,0.020515
ZTS,-0.009068,0.018116,-0.007967,-0.024529,0.004524,0.023557,-0.012210,-0.012917,-0.026895,0.001144,-0.021353,-0.035603,-0.026044,-0.007709,-0.003599,-0.006892,0.021980,0.005291,0.002137,-0.013470


In [11]:
pd.concat([risk_model['factor_returns'].head(2), risk_model['factor_returns'].tail(2)], axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2015-05-01 00:00:00+00:00,-0.231498,0.087009,-0.035185,-0.056721,0.006649,-0.014624,0.015049,-0.015601,-0.034371,0.031523,0.000828,-0.002096,0.017049,0.021042,0.019484,-0.058597,0.001738,-0.028662,0.026607,-0.074468
2015-05-04 00:00:00+00:00,-0.054719,0.007882,0.043880,0.020466,-0.049734,0.038801,-0.013299,0.006079,0.007780,-0.013627,0.000676,-0.017142,-0.028415,-0.011957,-0.016032,0.018925,-0.009592,0.018181,0.010830,-0.002648
2016-01-04 00:00:00+00:00,0.306592,-0.160879,0.008948,0.013268,0.031139,-0.058882,0.033337,-0.015162,-0.023391,-0.016096,0.097016,0.010075,-0.003835,0.040370,0.002952,-0.002508,-0.023565,0.020703,0.000560,-0.006553
2016-01-05 00:00:00+00:00,-0.029237,0.024917,0.105829,-0.051166,-0.021297,-0.008442,0.002659,-0.003693,0.021168,-0.058780,0.003526,-0.039558,-0.008585,-0.004922,-0.009306,0.006846,0.011186,0.006250,-0.037997,-0.006256


In [12]:
pd.concat([risk_model['factor_var_vector'].head(2), risk_model['factor_var_vector'].tail(2)], axis=0)

,0
0,13.101363
1,1.741896
18,0.201021
19,0.165043


In [13]:
pd.concat([risk_model['idiosyncratic_var_vector'].head(2), risk_model['idiosyncratic_var_vector'].tail(2)], axis=0)

,0
asset,
A,0.014894
AAL,0.029168
ZION,0.014803
ZTS,0.083648


## ML-Alpha

In [14]:
features_df = pd.read_csv(INPATH_ALPHA + INFILE1, parse_dates=['date'])
features_df['date'] = features_df['date'].dt.tz_localize('utc')
features_df.set_index(['date', 'asset'], inplace=True)
split = features_df.pop(split_col)
features = features_df.columns.tolist()
pd.concat([features_df.head(2), features_df.tail(2)],axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-12-28 00:00:00+00:00 ZION                                 0.773366   
                          ZTS                                 -0.944433   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-12-28 00:00:00+00:00 ZION               -0.188031   
                          ZTS                 0.209727   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-12-28 00:00:00+00:00 ZION                       1.325552 -0.965002   
                          ZTS                        0.441851  0.229762   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-12-28 00:00:00+00:00 ZION  -0.724635         0.014916        0.014731   
                          ZTS   -0.180275         0.014916        0.014731   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-12-28 00:00:00+00:00 ZION          0.183680        0.181479   
                          ZTS           0.183680        0.181479   

                                 volatility_120d  ...    sector_code_5  \
date                      asset                   ...                    
2013-01-03 00:00:00+00:00 A            -0.836546  ...                0   
                          AAL           1.639924  ...                0   
2015-12-28 00:00:00+00:00 ZION          0.547895  ...                1   
                          ZTS           0.413572  ...                0   

                                 sector_code_6  sector_code_7  sector_code_8  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-12-28 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_9  sector_code_10  qtr_1  qtr_2  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0               0      1      0   
                          AAL                0               0      1      0   
2015-12-28 00:00:00+00:00 ZION               0               0      0      0   
                          ZTS                0               0      0      0   

                                 qtr_3  qtr_4  
date                      asset                
2013-01-03 00:00:00+00:00 A          0      0  
                          AAL        0      0  
2015-12-28 00:00:00+00:00 ZION       0      1  
                          ZTS        0      1  

[4 rows x 33 columns]

In [15]:
valid_dates = split[split=='valid'].index.get_level_values(0)
valid_start = valid_dates[0].strftime('%Y-%m')
valid_end = valid_dates[-1].strftime('%Y-%m')

In [16]:
test_dates = split[split=='test'].index.get_level_values(0)
test_start = test_dates[0].strftime('%Y-%m-%d')
test_end = test_dates[-1].strftime('%Y-%m-%d')

In [17]:
train_valid_idx = split[(split=='train') | (split=='valid')].index.get_level_values(0).drop_duplicates()

In [18]:
X_train_valid = features_df.loc[(train_valid_idx, slice(None)), :]
pd.concat([X_train_valid.head(2), X_train_valid.tail(2)],axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-05-27 00:00:00+00:00 ZION                                 0.216057   
                          ZTS                                 -1.418772   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-05-27 00:00:00+00:00 ZION               -0.884221   
                          ZTS                 1.393649   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-05-27 00:00:00+00:00 ZION                      -0.078567 -1.085659   
                          ZTS                        1.378501  0.249987   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-05-27 00:00:00+00:00 ZION  -1.342789         0.012559        0.011522   
                          ZTS    0.378552         0.012559        0.011522   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-05-27 00:00:00+00:00 ZION          0.129420        0.104706   
                          ZTS           0.129420        0.104706   

                                 volatility_120d  ...    sector_code_5  \
date                      asset                   ...                    
2013-01-03 00:00:00+00:00 A            -0.836546  ...                0   
                          AAL           1.639924  ...                0   
2015-05-27 00:00:00+00:00 ZION          0.664252  ...                1   
                          ZTS          -0.321412  ...                0   

                                 sector_code_6  sector_code_7  sector_code_8  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-05-27 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_9  sector_code_10  qtr_1  qtr_2  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0               0      1      0   
                          AAL                0               0      1      0   
2015-05-27 00:00:00+00:00 ZION               0               0      0      1   
                          ZTS                0               0      0      1   

                                 qtr_3  qtr_4  
date                      asset                
2013-01-03 00:00:00+00:00 A          0      0  
                          AAL        0      0  
2015-05-27 00:00:00+00:00 ZION       0      0  
                          ZTS        0      0  

[4 rows x 33 columns]

In [19]:
with open(INPATH_ALPHA + MODEL_ALPHA, 'rb') as file:
    model = pickle.load(file)

In [20]:
pred_probas = model.predict_proba(features_df)
ml_alpha = mle_utils.get_pred_alpha(pred_probas)
ml_alpha = pd.Series(index=features_df.index, data=ml_alpha)
#ml_alpha[split_col] = split

In [21]:
alpha_facret = pd.read_csv(INPATH_ALPHA + INFILE3)

## Returns

In [ ]:
adv_win = cfg['models']['universe']['window']
adv_top = cfg['models']['universe']['adv_top']
print('Universe ADV window: {} and top threshold: {}'.format(adv_win, adv_top))

In [ ]:
# Data Bundle
os.environ['ZIPLINE_ROOT'] = bundle_path
ingest_func = bundles.csvdir.csvdir_equities(['daily'], BUNDLE_NAME)
bundles.register(BUNDLE_NAME, ingest_func)
print('Data Registered')

In [ ]:
universe = AverageDollarVolume(window_length=adv_win).top(adv_top) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(BUNDLE_NAME)
engine =  quant_helper.build_pipeline_engine(bundle_data, trading_calendar)

In [ ]:
universe_end_date =  pd.Timestamp(test_end, tz='UTC')
universe_start_date =  pd.Timestamp(test_start, tz='UTC')
print('Universe start: {} and end: {} dates'.format(universe_end_date, universe_start_date))

In [ ]:
universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers[:5]

In [ ]:
data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

In [ ]:
pricing = quant_helper.get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_start_date - pd.DateOffset(days=2),
        universe_end_date)
   
daily_returns = pricing .pct_change()[1:].fillna(0)
daily_returns.columns = [x.symbol for x in daily_returns.columns]
pd.concat([daily_returns.head(2), daily_returns.tail(2)], axis=0)

In [ ]:
volume  = quant_helper.get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_start_date - pd.DateOffset(days=44),
        universe_end_date, field='volume')
volume.head(2)

In [ ]:
test_dates[0].strftime('%Y-%m-%d')


In [ ]:
test_dates[-1].strftime('%Y-%m-%d')

In [ ]:
adv = volume.rolling(transcost_adv_win, axis=0).mean()
adv.columns = [x.symbol for x in adv.columns]
adv = np.clip(adv.iloc[transcost_adv_win:], a_min=transcost_adv_impute, a_max=np.inf)
adv.head(2)

In [ ]:
universe_tickers = [x.symbol for x in universe_tickers]